# Example 2: ITS_LIVE

**Description of ITS_LIVE dataset** 

- overview of what we'll do in example? 

In [1]:
import geopandas as gpd
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from shapely.geometry import Polygon
from shapely.geometry import Point
import cartopy.crs as ccrs
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy
import cartopy.feature as cfeature
import json
import s3fs

%config InlineBackend.figure_format='retina'

In [2]:
catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

In [3]:
url = catalog.iloc[4]['zarr_url']

In [7]:
def read_in_s3(http_url, chunks = 'auto'):
    '''this function takes an http url (from itslive catalog), converts to s3 url and returns the zarr datacube stored in the s3 bucket pointed to w/ url.
    will also set the crs based on the projection included in the attrs
    '''
    s3_url = http_url.replace('http','s3')
    s3_url = s3_url.replace('.s3.amazonaws.com','')
    print(s3_url)
    datacube = xr.open_dataset(s3_url, engine = 'zarr',
                                storage_options={'anon':True},
                                chunks = chunks)
    #set crs from attrs
    datacube = datacube.rio.write_crs(f"EPSG:{datacube.attrs['projection']}")
    return datacube


In [10]:
ds = xr.open_dataset('s3://its-live-data/datacubes/v02/S50W070/ITS_LIVE_vel_EPSG32718_G0120_X550000_Y4250000.zarr',
                     engine='zarr',
                     storage_options = {'anon':True},
                     chunks = 'auto')
    

In [11]:
ds

<xarray.Dataset>
Dimensions:                    (mid_date: 2302, y: 833, x: 833)
Coordinates:
  * mid_date                   (mid_date) datetime64[ns] 2018-10-13T14:39:09....
  * x                          (x) float64 5.001e+05 5.002e+05 ... 5.999e+05
  * y                          (y) float64 4.3e+06 4.3e+06 ... 4.2e+06 4.2e+06
Data variables: (12/54)
    acquisition_date_img1      (mid_date) datetime64[ns] dask.array<chunksize=(2302,), meta=np.ndarray>
    acquisition_date_img2      (mid_date) datetime64[ns] dask.array<chunksize=(2302,), meta=np.ndarray>
    autoRIFT_software_version  (mid_date) <U5 dask.array<chunksize=(2302,), meta=np.ndarray>
    chip_size_height           (mid_date, y, x) float32 dask.array<chunksize=(2302, 120, 120), meta=np.ndarray>
    chip_size_width            (mid_date, y, x) float32 dask.array<chunksize=(2302, 120, 120), meta=np.ndarray>
    date_center                (mid_date) datetime64[ns] dask.array<chunksize=(2302,), meta=np.ndarray>
    ...                         ...
    vy_error_mask              (mid_date) float64 dask.array<chunksize=(2302,), meta=np.ndarray>
    vy_error_modeled           (mid_date) float64 dask.array<chunksize=(2302,), meta=np.ndarray>
    vy_error_slow              (mid_date) float64 dask.array<chunksize=(2302,), meta=np.ndarray>
    vy_stable_shift            (mid_date) float64 dask.array<chunksize=(2302,), meta=np.ndarray>
    vy_stable_shift_mask       (mid_date) float64 dask.array<chunksize=(2302,), meta=np.ndarray>
    vy_stable_shift_slow       (mid_date) float64 dask.array<chunksize=(2302,), meta=np.ndarray>
Attributes: (12/18)
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               08-Jun-2022 17:51:09
    date_updated:               08-Jun-2022 17:51:09
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v02/S50W070/ITS_...
    skipped_granules:           s3://its-live-data/datacubes/v02/S50W070/ITS_...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image_pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

The dataset is already in a `x`,`y`, `time` cube, which is great. However, taking a look at the rest of the object you can see that this is still a somewhat unwieldy dataset, and there are ways we could organize it to better take advantage of xarray's functionality. 

The dataset contains many (53) `data_vars`, not all of which are independent (is this necessary?) observables. Since many of these could be described as metadata, and they are metadata attributes that we'd like to use later on, let's add them as `coordinate_vars` instead of `data_vars`. 

In [8]:
# check that mission_img1 == missing_img2 always
# add as non dim coord
# do the same w/ acq_img1_date, acq_img2_date -- adding non-dim datetime coordinate var
# I think all vars until v could be coords? 

What other tidying does this dataset need? 